참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import numpy.matlib as nm  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 05.002

### 문제에서 주어진 변수<br>Given Parameters

#### 각 구간 길이<br>Length of each section

In [ ]:
L_AB_m = 10.0
L_BD_m = 4.0

L_AD_m = L_AB_m + L_BD_m

#### 하중

In [ ]:
w_AB_N_m = -400e3
P_D_N = -1000e3

#### 단면

In [ ]:
# consider renaming variables
# tb_m -> b_m
# tt_m -> t_m
# ih_m -> h_w_m

tb_m = 150e-3
tt_m = 20e-3
ih_m = 200e-3

### A, B, C, D 지점의 x 좌표<br>x coorinates of A, B, C, D points
A 점에서 x=0 으로 한다.

In [ ]:
x_A_m = 0
x_B_m = x_A_m + L_AB_m
x_D_m = x_B_m + L_BD_m

### 반력

A점 중심의 모멘트는 분포하중, D점 하중, E점 반력이 각각 가하는 모멘트의 합이 된다.

$$
\int_{x=0}^{x=L_{AB}} wx \, dx+R_B L_{AB}+P_D L_{AD}=0\\
$$

In [ ]:
x, L_AB, w, R_B, P_D, L_AD = sy.symbols('x, L_AB, w, R_B, P_D, L_AD')

나중에 사용하기 위해 변수값을 표 형태로 지정한다.

In [ ]:
s_d = {L_AB: L_AB_m, 
       w: w_AB_N_m, 
       P_D: P_D_N, 
       L_AD: L_AD_m}
s_d

In [ ]:
moment_eq = sy.Eq(sy.integrate(w * x, (x, 0, L_AB)) 
              + R_B * L_AB + P_D * L_AD,
              0)
moment_eq

B점 반력을 구하면 다음과 같다.

In [ ]:
R_B_sol_list = sy.solve(moment_eq, R_B)
R_B_sol_list

B점에서의 반력에 변수값을 대입한 결과는 다음과 같다.

In [ ]:
R_B_N = float(R_B_sol_list[0].subs(s_d))
R_B_N

In [ ]:
R_B_kN = R_B_N * 1e-3
R_B_kN

A점에서의 반력은 수직방향 힘 평형 방정식으로부터 구할 수 있다.

A 점에서의 반력
$$
R_A + w L_{AB} + R_B + P_D = 0 \\
R_A = -w L_{AB} - R_B - P_D
$$

In [ ]:
R_A_N = - w_AB_N_m * L_AB_m - P_D_N - R_B_N
R_A_N

### 전단력선도

#### A-B 구간

In [ ]:
x_AB_m_array = np.linspace(x_A_m, x_B_m, 100+1)

A 지점에서의 반력과 분포하중을 반영한다.

$$V_{AB}=R_A+wx$$

In [ ]:
V_AB_N_array = R_A_N + w_AB_N_m * x_AB_m_array
V_AB_N_array[:5], V_AB_N_array[-5:], 

In [ ]:
plt.clf()
plt.fill_between(x_AB_m_array, V_AB_N_array)
plt.ylabel('V(N)')

A 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax = plt.axes()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.xlim(xmax=x_D_m)
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

#### B-D 구간

In [ ]:
x_BD_m_array = np.linspace(x_B_m, x_D_m, 40+1)

A 지점에서의 반력과 분포하중, B 지점 반력을 반영한다.

$$V_{BD}=R_A+wL_{AB}+R_B$$

In [ ]:
V_BD_N_array = (R_A_N + w_AB_N_m * L_AB_m + R_B_N) * np.ones_like(x_BD_m_array)

In [ ]:
plt.clf()
plt.fill_between(x_AB_m_array, V_AB_N_array)
plt.fill_between(x_BD_m_array, V_BD_N_array)
plt.ylabel('V(N)')

A 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax = plt.axes()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.2, head_length=0.05 * abs(R_A_N), fc='k', ec='k')

B 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax.arrow(x_B_m, V_AB_N_array[-1], 
         0, R_B_N * 0.9, 
         head_width=0.2, head_length=0.1 * abs(R_B_N), fc='k', ec='k')

D 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax.arrow(x_D_m, V_BD_N_array[-1], 
         0, P_D_N * 0.9, 
         head_width=0.2, head_length=0.1 * abs(P_D_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

### 굽힘모멘트 선도

A지점의 반력과 분포하중이 A-D 구간에 작용하는 모멘트를 반영한다.

$$M_{AD}=R_Ax+\frac{1}{2}wx^2$$

In [ ]:
M_AB_Nm_array = si.cumtrapz(V_AB_N_array, x_AB_m_array, initial=0)
M_BD_Nm_array = si.cumtrapz(V_BD_N_array, x_BD_m_array, initial=0) + M_AB_Nm_array[-1]

In [ ]:
plt.fill_between(x_AB_m_array, M_AB_Nm_array)
plt.fill_between(x_BD_m_array, M_BD_Nm_array)
plt.ylabel('M(Nm)')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

굽힘 모멘트 최대값을 찾는다.

In [ ]:
M_max_Nm = max(max(M_AB_Nm_array), max(M_BD_Nm_array))
M_max_Nm

최대값의 $x$ 좌표는 다음과 같이 알 수 있다.

In [ ]:
index_max_M = np.argmax(M_AB_Nm_array)
x_AB_m_array[index_max_M]

굽힘 모멘트 최소값을 찾는다.

In [ ]:
M_min_Nm = min(min(M_AB_Nm_array), min(M_BD_Nm_array))
M_min_Nm

최소값의 $x$ 좌표는 다음과 같이 알 수 있다.

In [ ]:
index_min_M = np.argmin(M_AB_Nm_array)
x_AB_m_array[index_min_M]

### 굽힘 응력

굽힘 응력과 굽힘 모멘트 사이에는 다음과 같은 관계가 있다. (p. 170, eq 5.4a)

$$\sigma_{max}=\frac{|M|_{max}c}{I}=\frac{|M|_{max}}{\frac{I}{c}}=\frac{|M|_{max}}{S}$$

#### 단면의 도심<br>Finding centroid of the section

##### 숫자값으로 구하기<br>Numerically

단면의 모멘트를 구하기 위해 단면의 도심을 구한다.

T 자 단면의 윗 부분 면적은 다음과 같다.

$$T_1 = tb \times tt$$

In [ ]:
T_1_m2 = tb_m * tt_m

T 자 단면의 아랫 부분 면적은 다음과 같다.

$$T_2 = ih \times tt$$

In [ ]:
T_2_m2 = ih_m * tt_m

윗 면을 기준으로 하면 도심의 위치 $c$는 다음과 같을 것이다.

$$(T_1+T_2)c_{up} = T_1\frac{tt}{2}+T_2\left(tt+\frac{ih}{2}\right)$$

$$c_{up} = \frac{T_1}{T_1+T_2}\frac{tt}{2}+\frac{T_2}{T_1+T_2}\left(\frac{2tt+ih}{2}\right)$$

$$c_{up} = \frac{1}{2(T_1+T_2)}[T_1tt+T_2(2tt+ih)]$$

In [ ]:
c_up_m = (T_1_m2 * tt_m + T_2_m2 * (2*tt_m + ih_m))/(2 * (T_1_m2 + T_2_m2))
c_up_m

아랫면을 기준으로 한다면 다음과 같을 것이다.

$$(T_1+T_2)c_{down} = T_1\left(\frac{tt}{2}+ih\right)+T_2\frac{ih}{2}$$

$$c_{down} = \frac{1}{2(T_1+T_2)}[T_1(tt+2ih)+T_2ih]$$

In [ ]:
c_down_m = (T_1_m2 * (tt_m + 2*ih_m) + T_2_m2 * ih_m)/(2 * (T_1_m2 + T_2_m2))
c_down_m

확인을 위해 두 값을 합한 다음 $tt+ih$ 와 비교한다. 이 값은 0에 가까와야 한다.

In [ ]:
(c_up_m + c_down_m - (tt_m + ih_m))/(tt_m + ih_m)

In [ ]:
assert (1e-7 > abs(c_up_m + c_down_m - (tt_m + ih_m))/(tt_m + ih_m))

##### sympy 로 구해보기<br>symbolically

단면의 치수 기호를 준비한다.

In [ ]:
b, t, h_web = sy.symbols('b, t, h_web', positive=True)

s_d.update({
        b: tb_m,
        t: tt_m,
        h_web: ih_m,
    })

플랜지 쪽 면적<br>Area of the flange

In [ ]:
A_flange = b * t

웨브 쪽 면적<br>Area of the web

In [ ]:
A_web = t * h_web

아래쪽 끝에서 부터 웨브 도심의 위치<br>Distance from the bottom to the centroid of web

In [ ]:
c_web = h_web / 2

아래쪽 끝에서 부터 플랜지 도심의 위치

In [ ]:
c_flange = h_web + t / 2

전체 면적의 도심의 위치를 구하기 위한 방정식<br>Equation to find where the centroid of the whole section is

In [ ]:
y_c = sy.symbols('y_c', real=True)
centroid_eq = sy.Eq(A_web * c_web + A_flange * c_flange, (A_flange + A_web) * y_c)

In [ ]:
centroid_eq

방정식을 풀어서 구한 도심의 위치<br>Centroid of the whole section by solving the equation

In [ ]:
y_c_sol = sy.solve(centroid_eq, y_c)[0]

값은 다음과 같이 확인 가능하다.<br>We can verify its value as follows.

In [ ]:
y_c_sym = y_c_sol.subs(s_d)

In [ ]:
y_c_sym

숫자 값으로 구한 결과와 비교한다.

In [ ]:
assert 1e-7 > abs((y_c_sym - c_down_m)/ y_c_sym)

#### 단면의 모멘트

##### `sympy` 로 적분

아랫 면을 기준으로 적분한다.

$$I=\int_{y=0}^{y=tt+ih}b(y)\cdot (y-c_{down})^2 \, dy$$

$$
b(y) = \begin{cases} 
    tt, & 0 \le y \le ih \\ 
    tb, & ih \le y \le ih + tt 
\end{cases}
$$

$$I=\int_{y=0}^{y=ih}tt \cdot (y-c_{down})^2 \, dy 
+ \int_{y=ih}^{y=ih+tt}tb \cdot (y-c_{down})^2 \, dy$$

In [ ]:
y, ih, tt, c, tb = sy.symbols('y, ih, tt, c, tb')

In [ ]:
s_d_i = {ih:200e-3, 
         tt:20e-3, 
         c:c_down_m, 
         tb:150e-3}

In [ ]:
I = sy.integrate(tt * (y-c)**2, (y, 0, ih)) + sy.integrate(tb * (y-c)**2, (y, ih, ih+tt))
sy.simplify(I)

In [ ]:
I_m4 = float(I.subs(s_d_i))
I_m4

##### 평행축 정리

어떤 단면의 면적, 모멘트가 각각 $A$, $I$ 이고, 도심의 좌표가 $y$ 일 때 어떤 다른 점 $c$ 에 대한 모멘트는 다음과 같다.

$$I_c=I+A(y-c)^2$$

윗 부분의 $c_{down}$ 에 대한 모멘트는 다음과 같다.

$$
\begin{align}
I_{1c}  &=I_1+A_1(y_1-c_{down})^2 \\
        &=\frac{tb \cdot tt^3}{12}+tb \cdot tt(ih+\frac{tt}{2} - c_{down})^2 
\end{align}
$$

In [ ]:
I1c = (tb_m * tt_m ** 3) / 12 + tb_m * tt_m * (ih_m + 0.5 * tt_m - c_down_m)**2
I1c

아랫 부분의 $c_{down}$ 에 대한 모멘트는 다음과 같다.

$$
\begin{align}
I_{2c}  &=I_2+A_2(y_2-c_{down})^2 \\
        &=\frac{tt \cdot ih^3}{12}+tt \cdot ih(\frac{ih}{2} - c_{down})^2 
\end{align}
$$

In [ ]:
I2c = (tt_m * ih_m ** 3) / 12 + tt_m * ih_m * (ih_m * 0.5 - c_down_m)**2
I2c

합치면 다음과 같을 것이다.

$$
\begin{align}
I&=I_{1c}+I_{2c} \\
&=tb \cdot tt \left[ \frac{tt^2}{12} + \left(ih+\frac{tt}{2} - c_{down}\right)^2 \right]\\
&+ih \cdot tt\left[ \frac{ih^2}{12}+(\frac{ih}{2} - c_{down})^2 \right]
\end{align}
$$

In [ ]:
I_c_m4 = I1c + I2c
I_c_m4

두 결과를 비교하면 다음과 같다. (이 결과도 0에 가까와야 한다)

In [ ]:
(I_c_m4 - I_m4) / I_m4

아래는 위 오차값의 크기를 검사한다

In [ ]:
assert (1e-7 > abs((I_c_m4 - I_m4)/I_m4))

같은 면적의 사각 단면의 경우 다음과 같을 것이다. ($m^4$)

In [ ]:
(35*200**3) / 12 * (1e-3)**4

#### 최대 응력

##### 모멘트 최대점

윗면 (MPa)

In [ ]:
- M_max_Nm * c_up_m / I_m4 *1e-6

아랫면 (MPa)

In [ ]:
- M_max_Nm * c_down_m / I_m4 *1e-6

##### 모멘트 최소점

윗면 (MPa)

In [ ]:
- M_min_Nm * c_up_m / I_m4 *1e-6

아랫면 (MPa)

In [ ]:
- M_min_Nm * c_down_m / I_m4 *1e-6

모멘트 최소점의 아랫면에서 최대 응력이 인장 상태로 발생하는 것을 알 수 있다.

In [ ]:
index_min_M = np.argmin(M_AB_Nm_array)
x_AB_m_array[index_min_M]

### 굽힘 응력 분포<br>Distribution of bending stress

굽힘 응력이 보의 길이 방향 `x` 위치별, 단면 안에서 중립면으로부터의 `y` 거리별로 어떻게 분포하는지 그림으로 나타내 보자.<br>
Let's visualize how bending stress is distributed across longitudinal `x` direction and vertical `y` direction.

`y` 배열을 더 높은 해상도로 새로 만든다.<br>Make a new `y` array with higher resolution.

In [ ]:
y_m_array = np.linspace(-c_down_m, c_up_m, 50 + 1)

구간별 `x` 배열을 이어 새로운 `x` 배열을 만든다.<br>Make a new `x` array by concatenating `x` arrays of each interval.

In [ ]:
x_m_array = np.concatenate((x_AB_m_array, x_BD_m_array))

`x` `y` 좌표 격자<br>`x` `y` coordinate grids

In [ ]:
y_m_grid, x_m_grid = np.meshgrid(y_m_array, x_m_array)

모멘트 배열도 어어서 만든다.<br>Concatenate moment arrays.

In [ ]:
M_Nm_array = np.concatenate((M_AB_Nm_array, M_BD_Nm_array))

모멘트 배열로 부터 모멘트 격자를 만든다.<br>Make moment grid from moment array.

In [ ]:
M_Nm_grid = nm.repmat(M_Nm_array.T, x_m_grid.shape[1], 1).T

준비된 격자의 크기가 모두 같은지 확인한다.<br>Assert if all grids are all in the same shape.

In [ ]:
assert(M_Nm_grid.shape == x_m_grid.shape == y_m_grid.shape)

굽힘모멘트와 `y` 거리, 단면의 `I_m4` 값을 이용하여 굽힘 법선응력을 계산한다.<br>Calculate bending normal stress using bending moment, `y` distance, and `I_m4` of the section.

In [ ]:
sigma_Pa_grid = M_Nm_grid * y_m_grid / (-I_m4)

굽힘응력을 그림으로 표시한다.<br>Plot bending stress.

In [ ]:
# https://matplotlib.org/gallery/images_contours_and_fields/pcolormesh_levels.html
plt.pcolormesh(x_m_grid, y_m_grid, sigma_Pa_grid * 1e-6, shading='flat', cmap='jet')
plt.xlabel('x(m)')
plt.ylabel('y(m)')
# https://stackoverflow.com/questions/15908371
cbar = plt.colorbar()
cbar.set_label('$\sigma$(MPa)')
plt.show()